In [219]:
import bs4 as bs
import cv2
import csv
import glob
import pytesseract
import os
import urllib.request
import httplib2

from PIL import Image
from sentence_transformers import SentenceTransformer, util
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification


In [169]:
def model_name_to_model_path(model_name):

    model_path = "/datasets/huggingface_hub/models--"+str(model_name)+"/snapshots/"

    dirs = os.listdir(model_path)
    if len(dirs) == 1:
        return os.path.join(model_path, dirs[0])
    else:
        raise ValueError(f"There is more than one file in the {model_path}")


## Download ads

In [170]:
url_google = "https://adstransparency.google.com/advertiser/AR13743371773506748417/creative/CR12193976855644078081?region=FR&platform=YOUTUBE"

In [121]:
url_1 = "https://library.tiktok.com/ads/detail/?ad_id=1759642641055778"
test_video_1 = "https://library.tiktok.com/api/v1/cdn/1739884858/video/aHR0cHM6Ly92NzcudGlrdG9rY2RuLmNvbS8wNGRlYmNiOWQ5NDA4N2Y2ZWI0MTc1NjY0ODIzMjBkNS82N2I0ZGRhNi92aWRlby90b3MvYWxpc2cvdG9zLWFsaXNnLXYtMDAwMC9vY3dRZGh3Q0JuSENOUWtVd29lNGc3b0ROQVFKR2VWYkRUQmtaZy8=/c1a21cbe-af88-4759-92a0-9294330bddea?a=475769&amp;bti=PDU2NmYwMy86&amp;ch=0&amp;cr=0&amp;dr=1&amp;cd=0%7C0%7C0%7C0&amp;cv=1&amp;br=268&amp;bt=134&amp;cs=0&amp;ds=1&amp;ft=.NpOcInz7ThYx.1OXq8Zmo&amp;mime_type=video_mp4&amp;qs=0&amp;rc=MzRmZjU5aTozaTdpNTU3aEBpM3J1Mzs6Zm8zajMzODYzNEAzYTU1Ll9gXmAxYF8uYzBfYSNeMTVgcjRfcTJgLS1kMC1zcw%3D%3D&amp;vvpl=1&amp;l=20250218132057B89936FF61D090DEBABE&amp;btag=e000b0000&amp;cc=13"

In [220]:
url_2 = "https://library.tiktok.com/ads/detail/?ad_id=1821860479764594"
test_video_2 = "https://library.tiktok.com/api/v1/cdn/1740063422/video/aHR0cHM6Ly92NzcudGlrdG9rY2RuLmNvbS9kYmJlYzE3NjI4ZGY5MDU1NWI4N2NiMmE4ZmU1NmFmYy82N2I3OTc0Yi92aWRlby90b3MvYWxpc2cvdG9zLWFsaXNnLXZlLTAwNTFjMDAxLXNnL29ZQXdER2NQZ0laQVViZVJHNGZlQ0xFSG9HVzh3eGc3cVhsaktBLw==/47fee5b8-1dc5-47fd-ac17-bc88eca76964?a=475769&amp;bti=PDU2NmYwMy86&amp;ch=0&amp;cr=0&amp;dr=1&amp;cd=0%7C0%7C0%7C0&amp;cv=1&amp;br=1822&amp;bt=911&amp;cs=0&amp;ds=1&amp;ft=.NpOcInz7Thkvr1OXq8Zmo&amp;mime_type=video_mp4&amp;qs=0&amp;rc=OGY0ZWdpaTM3ZmgzZTRlM0BpM3Q2M3g5cm1yeDMzODYzNEBjNmI1NmNfNS4xYC9iX2BeYSNrZmlyMmQ0ZTRgLS1kMC1zcw%3D%3D&amp;vvpl=1&amp;l=20250220145700A11D45138D8C960FCC5D&amp;btag=e00088000&amp;cc=13"

In [240]:
def download_videos(list_videos):

    for i in range(0,len(list_videos)):
        url = list_videos[i]
        h = httplib2.Http()
        resp, content = h.request(url)
        
        if not os.path.exists(videos):
            os.makedirs(videos)
            
        if resp.status == 200:
            output_path = os.path.join(videos, f"video_{i}.mp4") 
            urllib.request.urlretrieve(url, output_path)

In [241]:
list_videos = [test_video_1, test_video_2] 

In [242]:
download_videos(list_videos)

## Get frames from videos

In [214]:
def extract_frames(video_path, output_folder):
    
    video_name = os.path.splitext(os.path.basename(video_path))[0] 
    print(video_name)
    capture = cv2.VideoCapture(video_path)
    
    if not capture.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return
        
    frame_number = 0
    frame_skip = 3

    saved_frame_count = 0  


    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    while True:
        ret, frame = capture.read() 

        if not ret:
            break

        if frame_number % frame_skip == 0:
            frame_filename = os.path.join(output_folder, f"{video_name}_frame_{saved_frame_count}.jpg") 
            try:
                cv2.imwrite(frame_filename, frame)
                saved_frame_count += 1
                #print(f"Saved frame: {frame_filename}") #added print statement.
            except Exception as e:
                print(f"Error saving frame: {e}")
            saved_frame_count += 1 

        frame_number += 1 

    capture.release()

# Create Categories to analyze the video

In [ ]:
def categorize_frame(video_name, frame):
    try:
        text = pytesseract.image_to_string(frame)
        if text.strip():  # Check if the extracted text is not empty
            #print(text)
            category = "text or image and text"
        else:
            category = "image"
            #return "image"
    except pytesseract.TesseractNotFoundError:
        print("Error: Tesseract not found. Install it.")
        return "unknown" 
    except Exception as e: 
        print(f"Error during OCR: {e}")
        category = "error"
        return "unknown"

    return category, text



In [ ]:
test_videos = "./test_videos"
output_frames = "./output_frames"
csv_file_path = 'text_videos.csv' 

with open(csv_file_path, "a", newline="", encoding="utf-8") as csv_file:  
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['video_name', 'frame_number', 'category', 'text']) 

    for filename in os.listdir(test_videos):
        if filename.endswith((".mp4")):
            video_name_without_extension = os.path.splitext(filename)[0]
            if not os.path.exists(output_frames):
                os.makedirs(output_frames)
            for frame_name in os.listdir(os.path.join(output_frames, video_name_without_extension)):
                path = os.path.join(output_frames, video_name_without_extension,frame_name)
                category, text = categorize_frame(filename, path) # Pass writer to function
                writer.writerow([filename, frame_name, category, text])
       
           

In [215]:
def main():

    for video in os.listdir(os.path.join(test_videos)):
        video_path = os.path.join(test_videos, video)
        print(video_path)
        output_path = os.path.join(output_frames, video.split(".")[0])
        print(output_path)
        extract_frames(video_path, output_path)

In [ ]:
main()

In [163]:
model_name = "openai--clip-vit-base-patch32"
model_name_lab = model_name_to_model_path(model_name)

In [165]:

processor = AutoProcessor.from_pretrained(model_name_lab)
model = AutoModelForZeroShotImageClassification.from_pretrained(model_name_lab)

In [167]:
image_names = list(glob.glob('output_frames/*.jpg'))

In [ ]:
image_names = list(glob.glob('output_frames/test_video_1/*.jpg'))
print("Images:", len(image_names))
encoded_image = model.encode([Image.open(filepath) for filepath in image_names], batch_size=128, convert_to_tensor=True, show_progress_bar=True)

# Now we run the clustering algorithm. This function compares images aganist 
# all other images and returns a list with the pairs that have the highest 
# cosine similarity score
processed_images = util.paraphrase_mining_embeddings(encoded_image)

In [ ]:


# Load the OpenAI CLIP Model


# Next we compute the embeddings
# To encode an image, you can use the following code:
# from PIL import Image
# encoded_image = model.encode(Image.open(filepath))
image_names = list(glob.glob('output_frames/test_video_1/*.jpg'))
print("Images:", len(image_names))
encoded_image = model.encode([Image.open(filepath) for filepath in image_names], batch_size=128, convert_to_tensor=True, show_progress_bar=True)

# Now we run the clustering algorithm. This function compares images aganist 
# all other images and returns a list with the pairs that have the highest 
# cosine similarity score
processed_images = util.paraphrase_mining_embeddings(encoded_image)
NUM_SIMILAR_IMAGES = 10 

# =================
# DUPLICATES
# =================
print('Finding duplicate images...')
# Filter list for duplicates. Results are triplets (score, image_id1, image_id2) and is scorted in decreasing order
# A duplicate image will have a score of 1.00
# It may be 0.9999 due to lossy image compression (.jpg)
duplicates = [image for image in processed_images if image[0] >= 0.999]

# Output the top X duplicate images
for score, image_id1, image_id2 in duplicates[0:NUM_SIMILAR_IMAGES]:
    print("\nScore: {:.3f}%".format(score * 100))
    print(image_names[image_id1])
    print(image_names[image_id2])

# =================
# NEAR DUPLICATES
# =================
print('Finding near duplicate images...')
# Use a threshold parameter to identify two images as similar. By setting the threshold lower, 
# you will get larger clusters which have less similar images in it. Threshold 0 - 1.00
# A threshold of 1.00 means the two images are exactly the same. Since we are finding near 
# duplicate images, we can set it at 0.99 or any number 0 < X < 1.00.
threshold = 0.99
near_duplicates = [image for image in processed_images if image[0] < threshold]

for score, image_id1, image_id2 in near_duplicates[0:NUM_SIMILAR_IMAGES]:
    print("\nScore: {:.3f}%".format(score * 100))
    print(image_names[image_id1])
    print(image_names[image_id2])